In [1]:
import torch
if torch.cuda.is_available:
  device = 'cuda'

In [2]:
class ConvertToRGB(object):
  def __call__(self, img):
    if img.mode != 'RGB':
      img.convert('RGB')
    return img

In [3]:
from torchvision import transforms
transform = transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [4]:
from torchvision import datasets
data_dir = '/content/drive/MyDrive/Images/Maize'
data = datasets.ImageFolder(data_dir, transform=transform)
data

Dataset ImageFolder
    Number of datapoints: 2541
    Root location: /content/drive/MyDrive/Images/Maize
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [5]:
from torch.utils.data import DataLoader
dloader = DataLoader(data, 32)
print(next(iter(dloader))[0].shape)

torch.Size([32, 3, 224, 224])


In [6]:
def mean_std(loader, max_batches=30): # max_batches for speed purposel
    sum_, sum_sq, count = 0, 0, 0

    for i, (data, _) in enumerate(loader):
        if i == max_batches:
            break
        batch_size, channels, height, width = data.shape
        data = data.view(batch_size, channels, -1)
        sum_ += data.sum(dim=[0, 2])
        sum_sq += (data ** 2).sum(dim=[0, 2])
        count += batch_size * height * width

    mean = sum_ / count
    std = (sum_sq / count - mean ** 2).sqrt()
    return mean, std

mean, std = mean_std(dloader)

In [12]:
transform_norm = transforms.Compose([
    ConvertToRGB(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1)),
    transforms.ToTensor(),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Normalize(mean=mean, std=std)
])

In [13]:
data_norm = datasets.ImageFolder(data_dir, transform=transform_norm)
nloader = DataLoader(data_norm, 32)

In [14]:
from torch.utils.data import random_split
g = torch.Generator().manual_seed(26)

train, val = random_split(data_norm, [0.75, 0.25], generator=g)

tloader = DataLoader(train, 32, shuffle=True, generator=g)
vloader = DataLoader(val, 32, generator=g)

In [15]:
import torch.nn as nn
model = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.MaxPool2d(kernel_size=(2, 2), stride=2),

    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2, 2), stride=2),

    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=(2, 2), stride=2),

    nn.Flatten(),
    nn.Linear(in_features=50176, out_features=2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=2048, out_features=512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=5)
)

In [16]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
model.to(device)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=50176, out_features=2048, bias=True)
  (12): ReLU()
  (13): Dropout(p=0.5, inplace=False)
  (14): Linear(in_features=2048, out_features=512, bias=True)
  (15): ReLU()
  (16): Dropout(p=0.5, inplace=False)
  (17): Linear(in_features=512, out_features=5, bias=True)
)

In [17]:
pip install torchinfo

In [18]:
from torchinfo import summary
summary(model, input_size=(32, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 5]                   --
├─Conv2d: 1-1                            [32, 16, 224, 224]        448
├─ReLU: 1-2                              [32, 16, 224, 224]        --
├─Dropout: 1-3                           [32, 16, 224, 224]        --
├─MaxPool2d: 1-4                         [32, 16, 112, 112]        --
├─Conv2d: 1-5                            [32, 32, 112, 112]        4,640
├─ReLU: 1-6                              [32, 32, 112, 112]        --
├─MaxPool2d: 1-7                         [32, 32, 56, 56]          --
├─Conv2d: 1-8                            [32, 64, 56, 56]          18,496
├─ReLU: 1-9                              [32, 64, 56, 56]          --
├─MaxPool2d: 1-10                        [32, 64, 28, 28]          --
├─Flatten: 1-11                          [32, 50176]               --
├─Linear: 1-12                           [32, 2048]                102,762,49

In [19]:
from tqdm.notebook import tqdm
def train(model, train_loader, val_loader, loss_fn, optimizer, device, epochs):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        training_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            training_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

        avg_train_loss = training_loss / len(train_loader.dataset)
        train_accuracy = train_correct / train_total * 100

        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = val_correct / val_total * 100

        print(
            f"Epoch {epoch+1}/{epochs} "
            f"- Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2f}% "
            f"- Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%"
        )

In [20]:
train(model, tloader, vloader, loss_fn, optimizer, device, epochs=5)

Training Epoch 1/5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/5 - Train Loss: 1.7896 | Train Acc: 21.93% - Val Loss: 1.5842 | Val Acc: 28.50%


Training Epoch 2/5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 4/5 - Train Loss: 0.9524 | Train Acc: 59.60% - Val Loss: 1.0944 | Val Acc: 51.97%


Training Epoch 5/5:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 5/5 - Train Loss: 0.8582 | Train Acc: 65.84% - Val Loss: 0.7868 | Val Acc: 69.92%


In [21]:
train(model, tloader, vloader, loss_fn, optimizer, device, epochs=3)

Training Epoch 1/3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/3 - Train Loss: 0.7738 | Train Acc: 70.88% - Val Loss: 0.8626 | Val Acc: 65.67%


Training Epoch 2/3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 2/3 - Train Loss: 0.7342 | Train Acc: 72.14% - Val Loss: 0.7902 | Val Acc: 69.61%


Training Epoch 3/3:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 3/3 - Train Loss: 0.6461 | Train Acc: 75.39% - Val Loss: 0.6730 | Val Acc: 74.80%


In [23]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Model/Maize_CNN.pth')

In [29]:
from PIL import Image
import pandas as pd
def classifier(path):
    image = Image.open(path)
    transformed = transform_norm(image)
    unsqueezed = transformed.unsqueeze(0)
    image_cuda = unsqueezed.to(device)

    model.eval()
    with torch.no_grad():
        model_raw = model(image_cuda)
        confidence = torch.nn.functional.softmax(model_raw, dim=1)

    conf_df = pd.DataFrame(confidence.tolist())
    conf_df.columns = data.classes
    predicted_class = conf_df.idxmax(axis=1).item()
    return predicted_class

In [31]:
classification = classifier('/content/drive/MyDrive/Images/Maize/Healthy/healthy (1).jpeg')

print(f"Predicted class: {classification}")

Predicted class: Healthy


In [33]:
cards = [
    '/content/drive/MyDrive/Images/Maize/Healthy/healthy (107).jpeg',
    '/content/drive/MyDrive/Images/Maize/Healthy/healthy (116).jpeg',
    '/content/drive/MyDrive/Images/Maize/Yellow/yellow (101).jpeg',
    '/content/drive/MyDrive/Images/Maize/Yellow/yellow (110).jpeg',
    '/content/drive/MyDrive/Images/Maize/Mosaic/mosaic (101).jpeg',
    '/content/drive/MyDrive/Images/Maize/Mosaic/mosaic (111).jpeg',
    '/content/drive/MyDrive/Images/Maize/RedRot/redrot (100).jpeg',
    '/content/drive/MyDrive/Images/Maize/RedRot/redrot (112).jpeg',
    '/content/drive/MyDrive/Images/Maize/Rust/rust (101).jpeg',
    '/content/drive/MyDrive/Images/Maize/Rust/rust (123).jpeg',
    '/content/drive/MyDrive/Images/Maize/Rust/rust (120).jpeg'
]

for card in cards:
    classified = classifier(card)
    print(f"Predicted class: {classified}")

Predicted class: Healthy
Predicted class: Healthy
Predicted class: Yellow
Predicted class: Yellow
Predicted class: Mosaic
Predicted class: Mosaic
Predicted class: RedRot
Predicted class: RedRot
Predicted class: Healthy
Predicted class: Yellow
Predicted class: Mosaic
